In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import chime
from sklearn.metrics import roc_auc_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [2]:
%load_ext chime

# Halving search on the complex ensemble
Kaggle Playground Series - Season 3, Episode 7

https://www.kaggle.com/competitions/playground-series-s3e7/overvie



# Data preparation

In [3]:
train = pd.read_csv('./data/train.csv', index_col='id'); train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
id,,,,,,,,,,,,,,,,,,
0,2,0,0,2,1,0,0,9,2018,1,14,1,1,11,0,67.50,0,0
1,2,0,1,2,0,0,0,117,2018,7,29,0,0,0,0,72.25,0,0
2,2,0,0,1,0,0,0,315,2018,12,2,0,0,0,0,52.00,0,0
3,1,0,0,2,1,0,0,32,2018,12,1,1,0,0,0,56.00,0,0
4,2,0,1,0,0,0,0,258,2018,10,16,0,0,0,0,100.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42095,3,0,0,4,0,0,1,160,2018,12,30,1,0,0,0,140.00,2,1
42096,2,0,0,3,0,0,0,34,2017,9,23,0,0,0,0,224.67,0,0
42097,2,0,0,2,2,0,0,292,2018,7,21,0,0,0,0,96.00,0,0


In [4]:
X_train = train.drop(columns='booking_status')
y_train = train.booking_status

In [5]:
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), 
                       columns=X_train.columns, 
                       index=X_train.index)
X_train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
0,0.151040,-0.313454,-0.998814,-0.278849,1.294624,-0.160945,-0.515173,-1.170469,0.409661,-2.330398,-0.214091,0.428551,5.766758,33.699120,-0.101479,-0.998052,-0.737691
1,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,0.161740,0.409661,-0.209778,1.473487,-1.149928,-0.173408,-0.060506,-0.101479,-0.870153,-0.737691
2,0.151040,-0.313454,-0.998814,-0.979466,-0.407020,-0.160945,-0.515173,2.604122,0.409661,1.557404,-1.564154,-1.149928,-0.173408,-0.060506,-0.101479,-1.415406,-0.737691
3,-1.753925,-0.313454,-0.998814,-0.278849,1.294624,-0.160945,-0.515173,-0.886758,0.409661,1.557404,-1.676659,0.428551,-0.173408,-0.060506,-0.101479,-1.307702,-0.737691
4,0.151040,-0.313454,0.130259,-1.680084,-0.407020,-0.160945,-0.515173,1.901012,0.409661,0.850531,0.010919,-1.149928,-0.173408,-0.060506,-0.101479,-0.122955,-0.737691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42095,2.056005,-0.313454,-0.998814,1.122386,-0.407020,-0.160945,0.685890,0.692156,0.409661,1.557404,1.585992,0.428551,-0.173408,-0.060506,-0.101479,0.954088,1.842849
42096,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-0.862087,-2.441040,0.497095,0.798456,-1.149928,-0.173408,-0.060506,-0.101479,3.233919,-0.737691
42097,0.151040,-0.313454,-0.998814,-0.278849,2.996268,-0.160945,-0.515173,2.320411,0.409661,-0.209778,0.573445,-1.149928,-0.173408,-0.060506,-0.101479,-0.230659,-0.737691


# Best models
Assembling data from tuning individual classifiers

### Random forest

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
best_params_rf =\
{'max_depth': 40,
 'max_features': 'sqrt',
 'max_samples': 1.0,
 'min_samples_split': 40,
 'n_estimators': 2250}

In [8]:
random_forest = RandomForestClassifier(n_jobs=-1,
                                       random_state=8,
                                       bootstrap=True,
                                      **best_params_rf)

In [9]:
best_score_rf = 0.8911929904636795

### XGBoost

In [10]:
from xgboost import XGBClassifier

In [11]:
best_params_xgb =\
{'learning_rate': 0.3,
 'max_depth': 3,
 'n_estimators': 700,
 'reg_alpha': 0.0005,
 'reg_lambda': 0.2}

In [12]:
xgboost_clf = XGBClassifier(random_state=8,
                         max_leaves=0,
                         n_jobs=-1,
                         **best_params_xgb)

In [13]:
best_score_xgb = 0.8983333718679651

### KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
best_params_knn = {'n_neighbors': 60, 'p': 1, 'weights': 'uniform'}

In [16]:
knn = KNeighborsClassifier(algorithm='auto',
                           metric='minkowski',
                           n_jobs=-1,
                          **best_params_knn)

In [17]:
best_score_knn = 0.8636631176440875

### AdaBoost

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [19]:
best_params_base = {'max_depth': 25,
 'max_features': 'sqrt',
 'min_samples_split': 150}

In [20]:
best_params_ada = {'learning_rate': 0.0005, 'n_estimators': 900}

In [21]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=8,
                                                                   **best_params_base),
                              random_state=8,
                              **best_params_ada)

In [22]:
best_score_ada = 0.8900577986404123

## Stacking  classifiers

### Final estimator: XGBoost

Without KNN

In [23]:
from sklearn.ensemble import StackingClassifier

In [26]:
stack_3 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('ada', ada)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=4,
)

params = {
    'final_estimator__n_estimators': [25, 70, 200],
    'final_estimator__max_depth': [2, 4, 6],
    'final_estimator__learning_rate': [0.01, 0.1, 1],
    'final_estimator__reg_alpha': [0, 0.1, 0.5],
    'final_estimator__reg_lambda': [0, 0.1, 0.5]
}

In [34]:
params = {
    'final_estimator__n_estimators': [50, 70, 125],
    'final_estimator__max_depth': [2, 3],
    'final_estimator__learning_rate': [0.05, 0.1, 0.5],
    'final_estimator__reg_alpha': [0.05, 0.1, 0.2],
    'final_estimator__reg_lambda': [0, 0.05]
}

In [35]:
search = HalvingGridSearchCV(
    estimator=stack_3,
    param_grid=params,
    factor=3,
    resource='n_samples',
    max_resources='auto',
    min_resources=2000,
    aggressive_elimination=True,
    cv=4,
    scoring='roc_auc',
    refit=True, 
    return_train_score=False,
    random_state=8, 
    n_jobs=-1,
    verbose=3
)

In [36]:
%%time
%%chime
search.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 3
min_resources_: 2000
max_resources_: 42100
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 108
n_resources: 2000
Fitting 4 folds for each of 108 candidates, totalling 432 fits
[CV 2/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0;, score=0.867 total time=  31.9s
[CV 1/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0.05;, score=0.858 total time=  32.8s
[CV 1/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.858 total time=  33.7s
[CV 2/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_est

[CV 1/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0;, score=0.858 total time=  32.1s
[CV 2/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0.05;, score=0.868 total time=  32.9s
[CV 2/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.868 total time=  32.9s
[CV 1/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.05;, score=0.858 total time=  32.9s
[CV 1/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_esti

[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0;, score=0.871 total time=  32.7s
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0.05;, score=0.871 total time=  35.5s
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.871 total time=  36.2s
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.05;, score=0.871 total time=  35.4s
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_esti

[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0.05;, score=0.871 total time=  33.9s
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.871 total time=  34.0s
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.05;, score=0.871 total time=  34.5s
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0;, score=0.871 total time=  34.0s
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_estimator_

[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.05;, score=0.867 total time=  37.2s
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0;, score=0.867 total time=  36.3s
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0.05;, score=0.867 total time=  35.8s
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0;, score=0.867 total time=  35.3s
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator

[CV 1/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.05;, score=0.839 total time=  34.8s
[CV 1/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0;, score=0.842 total time=  35.0s
[CV 1/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0.05;, score=0.840 total time=  35.6s
[CV 1/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0;, score=0.834 total time=  35.1s
[CV 1/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator

[CV 4/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0.05;, score=0.859 total time=  36.0s
[CV 4/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0;, score=0.856 total time=  36.9s
[CV 4/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0.05;, score=0.852 total time=  35.8s
[CV 4/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.851 total time=  35.3s
[CV 4/4] END final_estimator__learning_rate=0.5, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator

[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0;, score=0.853 total time=  33.9s
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0;, score=0.854 total time=  34.2s
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0.05;, score=0.853 total time=  34.0s
[CV 1/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=3, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.855 total time=  33.8s
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__r

[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=3, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.864 total time=  37.1s
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.866 total time=  36.5s
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.05;, score=0.866 total time=  37.3s
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=125, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0;, score=0.866 total time=  36.3s
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=125, final_estimator__reg_alpha=0.05, final_estimato

CPU times: user 3min 13s, sys: 1.5 s, total: 3min 14s
Wall time: 1h 44min 33s


In [37]:
search.best_params_

{'final_estimator__learning_rate': 0.05,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 70,
 'final_estimator__reg_alpha': 0.2,
 'final_estimator__reg_lambda': 0}

In [38]:
search.best_score_

0.889242813142243

In [39]:
results = pd.DataFrame(search.cv_results_)
results

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_final_estimator__learning_rate,param_final_estimator__max_depth,param_final_estimator__n_estimators,param_final_estimator__reg_alpha,param_final_estimator__reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0,2000,31.703858,0.367575,0.597099,0.016182,0.05,2,50,0.05,0,"{'final_estimator__learning_rate': 0.05, 'fina...",0.857675,0.867436,0.860456,0.871078,0.864162,0.005347,34
1,0,2000,32.971102,1.040498,0.641650,0.083589,0.05,2,50,0.05,0.05,"{'final_estimator__learning_rate': 0.05, 'fina...",0.857742,0.867597,0.860456,0.871181,0.864244,0.005385,25
2,0,2000,33.860196,1.264930,0.600233,0.020440,0.05,2,50,0.1,0,"{'final_estimator__learning_rate': 0.05, 'fina...",0.857642,0.867597,0.860456,0.871181,0.864219,0.005415,28
3,0,2000,33.543728,1.054952,0.667990,0.031637,0.05,2,50,0.1,0.05,"{'final_estimator__learning_rate': 0.05, 'fina...",0.857642,0.867487,0.860456,0.871130,0.864179,0.005382,31
4,0,2000,33.979441,0.721801,0.655109,0.033477,0.05,2,50,0.2,0,"{'final_estimator__learning_rate': 0.05, 'fina...",0.857659,0.867681,0.859661,0.871095,0.864024,0.005544,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,3,6000,59.629361,1.852057,1.065802,0.090915,0.05,2,70,0.2,0.05,"{'final_estimator__learning_rate': 0.05, 'fina...",0.876949,0.880525,0.876672,0.883429,0.879394,0.002781,6
158,3,6000,60.779199,3.140207,1.004231,0.058455,0.05,2,70,0.05,0.05,"{'final_estimator__learning_rate': 0.05, 'fina...",0.876955,0.880571,0.876780,0.883511,0.879454,0.002788,3
159,3,6000,59.196789,0.661367,0.854837,0.064713,0.05,2,70,0.2,0,"{'final_estimator__learning_rate': 0.05, 'fina...",0.876992,0.880606,0.876672,0.883493,0.879441,0.002804,4
160,4,18000,152.838564,0.372389,2.519396,0.101816,0.05,2,70,0.2,0,"{'final_estimator__learning_rate': 0.05, 'fina...",0.889445,0.892020,0.888349,0.887157,0.889243,0.001796,1


In [40]:
results.mean_test_score.describe()

count    162.000000
mean       0.857063
std        0.011851
min        0.823255
25%        0.854186
50%        0.860777
75%        0.864020
max        0.889243
Name: mean_test_score, dtype: float64

[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0;, score=0.873 total time=  28.3s
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.884 total time= 1.0min
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.2, final_estimator__reg_lambda=0.05;, score=0.883 total time= 1.1min
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.05, final_estimator__reg_lambda=0.05;, score=0.884 total time= 1.1min
[CV 4/4] END final_estimator__learning_rate=0.05, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.2, final_estim

Performance has not improved over individual classifiers

### With KNN

In [55]:
stack_4 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('knn', knn),
        ('ada', ada)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

params = {
    'final_estimator__n_estimators': [50, 70, 100],
    'final_estimator__max_depth': [2, 4, 6],
    'final_estimator__learning_rate': [0.01, 0.1, 0.5],
    'final_estimator__reg_alpha': [0.01, 0.1, 0.5],
    'final_estimator__reg_lambda': [0.01, 0.1, 0.5]
}

params = {
    'final_estimator__n_estimators': [10, 30, 50, 60],
    'final_estimator__max_depth': [2, 3],
    'final_estimator__learning_rate': [0.05, 0.1, 0.2],
    'final_estimator__reg_alpha': [0.05, 0.1, 0.2],
    'final_estimator__reg_lambda': [0.005, 0.01, 0.05]
}

In [74]:
params = {
    'final_estimator__n_estimators': [8, 10, 15],
    'final_estimator__max_depth': [3],
    'final_estimator__learning_rate': [0.15, 0.2, 0.25],
    'final_estimator__reg_alpha': [0.02, 0.03, 0.04],
    'final_estimator__reg_lambda': [0.04, 0.05, 0.06]
}

In [75]:
search_4 = HalvingGridSearchCV(
    estimator=stack_4,
    param_grid=params,
    factor=3,
    resource='n_samples',
    max_resources='auto',
    min_resources=2000,
    aggressive_elimination=True,
    cv=5,
    scoring='roc_auc',
    refit=True, 
    return_train_score=False,
    random_state=8, 
    n_jobs=-1,
    verbose=1
)

In [76]:
%%time
%%chime
search_4.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 3
min_resources_: 2000
max_resources_: 42100
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 81
n_resources: 2000
Fitting 5 folds for each of 81 candidates, totalling 405 fits
----------
iter: 1
n_candidates: 27
n_resources: 2000
Fitting 5 folds for each of 27 candidates, totalling 135 fits
----------
iter: 2
n_candidates: 9
n_resources: 2000
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 3
n_candidates: 3
n_resources: 6000
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 1
n_resources: 18000
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/home/denis/MyCode/Git/Portfolio/Kaggle/Playground_07_03/env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 4min 43s, sys: 7.15 s, total: 4min 50s
Wall time: 2h 3min 41s


In [77]:
search_4.best_params_

{'final_estimator__learning_rate': 0.2,
 'final_estimator__max_depth': 3,
 'final_estimator__n_estimators': 10,
 'final_estimator__reg_alpha': 0.02,
 'final_estimator__reg_lambda': 0.05}

In [78]:
search_4.best_score_

0.8889905546090582

In [73]:
pd.Series(search_4.cv_results_['mean_test_score']).describe()

count    162.000000
mean       0.866409
std        0.003708
min        0.860720
25%        0.864981
50%        0.866334
75%        0.867209
max        0.888991
dtype: float64

### Results so far
The best result is achieved when KNN is included in a stacked classifier with XGBoost as the final estimator, although it is computationally expensive. I will try to add logistic regression as well

### With KNN and logistic regression

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
best_params_lr = {'C': 0.1, 'max_iter': 50, 'penalty': 'l2'}
lr = LogisticRegression(class_weight='balanced',
                        random_state=8,
                        solver='liblinear',
                        **best_params_lr)

In [26]:
stack_5 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('knn', knn),
        ('ada', ada),
        ('lr', lr)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

params = {
    'final_estimator__n_estimators': [25, 70, 150],
    'final_estimator__max_depth': [2, 4, 6],
    'final_estimator__learning_rate': [0.01, 0.1, 0.5],
    'final_estimator__reg_alpha': [0, 0.1, 0.5],
    'final_estimator__reg_lambda': [0, 0.1, 0.5]
}

params = {
    'final_estimator__n_estimators': [15, 25, 50],
    'final_estimator__max_depth': [2, 3],
    'final_estimator__learning_rate': [0.05, 0.1, 0.3],
    'final_estimator__reg_alpha': [0.05, 0.1, 0.3],
    'final_estimator__reg_lambda': [0.05, 0.1, 0.3]
}

params = {
    'final_estimator__n_estimators': [10, 15, 20],
    'final_estimator__max_depth': [2],
    'final_estimator__learning_rate': [0.2, 0.3, 0.4],
    'final_estimator__reg_alpha': [0.02, 0.05, 0.7],
    'final_estimator__reg_lambda': [0.02, 0.05, 0.7]
}

In [49]:
params = {
    'final_estimator__n_estimators': [7, 10, 12],
    'final_estimator__max_depth': [2],
    'final_estimator__learning_rate': [0.25, 0.3, 0.35],
    'final_estimator__reg_alpha': [0.02, 0.03],
    'final_estimator__reg_lambda': [0.03, 0.05, 0.6]
}

In [50]:
search_5 = HalvingGridSearchCV(
    estimator=stack_5,
    param_grid=params,
    factor=3,
    resource='n_samples',
    max_resources='auto',
    min_resources=2000,
    aggressive_elimination=True,
    cv=5,
    scoring='roc_auc',
    refit=True, 
    return_train_score=False,
    random_state=8, 
    n_jobs=-1,
    verbose=3
)

In [51]:
%%time
%%chime
search_5.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 3
min_resources_: 2000
max_resources_: 42100
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 2000
Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 3/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.03;, score=0.845 total time=  39.5s
[CV 1/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.857 total time=  41.4s
[CV 1/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.6;, score=0.857 total time=  40.0s
[CV 4/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_es

[CV 2/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.03;, score=0.875 total time=  39.5s
[CV 2/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.875 total time=  40.6s
[CV 5/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.877 total time=  40.9s
[CV 5/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.6;, score=0.877 total time=  40.8s
[CV 4/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.03, final_e

[CV 4/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.03;, score=0.882 total time=  40.9s
[CV 3/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.843 total time=  44.1s
[CV 2/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.6;, score=0.875 total time=  42.3s
[CV 1/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.857 total time=  43.6s
[CV 5/5] END final_estimator__learning_rate=0.25, final_estimator__max_depth=2, final_estimator__n_estimators=7, final_estimator__reg_alpha=0.03, final_e

[CV 3/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.03;, score=0.849 total time=  43.7s
[CV 2/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.876 total time=  42.5s
[CV 1/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.6;, score=0.859 total time=  42.2s
[CV 5/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.6;, score=0.882 total time=  43.8s
[CV 4/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, final_es

[CV 1/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.860 total time=  43.3s
[CV 5/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.883 total time=  42.7s
[CV 4/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.6;, score=0.883 total time=  42.3s
[CV 3/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.849 total time=  43.7s
[CV 2/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, final_e

[CV 1/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.858 total time=  45.3s
[CV 5/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.880 total time=  44.6s
[CV 4/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.883 total time=  45.6s
[CV 3/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.851 total time=  44.2s
[CV 2/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, fi

/home/denis/MyCode/Git/Portfolio/Kaggle/Playground_07_03/env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 3/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.851 total time=  43.3s
[CV 2/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.877 total time=  43.8s
[CV 1/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.858 total time=  43.7s
[CV 5/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.880 total time=  43.6s
[CV 4/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, fi

[CV 2/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.02, final_estimator__reg_lambda=0.05;, score=0.877 total time=  42.5s
[CV 1/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.858 total time=  42.2s
[CV 5/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.881 total time=  43.0s
[CV 4/5] END final_estimator__learning_rate=0.35, final_estimator__max_depth=2, final_estimator__n_estimators=10, final_estimator__reg_alpha=0.03, final_estimator__reg_lambda=0.03;, score=0.885 total time=  42.1s
[CV 3/5] END final_estimator__learning_rate=0.3, final_estimator__max_depth=2, final_estimator__n_estimators=12, final_estimator__reg_alpha=0.03, fi

In [52]:
search_5.best_params_

{'final_estimator__learning_rate': 0.3,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 12,
 'final_estimator__reg_alpha': 0.03,
 'final_estimator__reg_lambda': 0.03}

In [53]:
search_5.best_score_

0.888985558921776

In [54]:
pd.Series(search_5.cv_results_['mean_test_score']).describe()

count    80.000000
mean      0.870552
std       0.004120
min       0.867088
25%       0.868628
50%       0.869450
75%       0.870594
max       0.888986
dtype: float64

Addition of logistic regression did not improve results

# Prediction

In [79]:
test = pd.read_csv('./data/test.csv', index_col='id'); test

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
42100,3,0,1,4,0,0,1,111,2018,12,5,1,0,0,0,115.60,2
42101,2,0,0,3,0,0,0,22,2017,10,21,0,0,0,0,85.00,0
42102,2,2,0,1,0,0,3,18,2018,8,10,1,0,0,0,240.00,1
42103,2,0,0,3,0,0,0,88,2018,5,30,0,0,0,0,80.75,0
42104,1,0,0,2,2,0,0,7,2018,9,21,1,0,0,0,144.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70163,2,0,1,2,0,0,0,315,2018,9,30,1,0,0,0,160.00,0
70164,2,0,2,2,0,0,0,81,2018,3,25,0,0,0,0,65.00,1
70165,2,0,1,2,0,0,0,40,2018,10,22,0,0,0,0,85.00,0


In [80]:
X_test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)
X_test

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
42100,2.056005,-0.313454,0.130259,1.122386,-0.407020,-0.160945,0.685890,0.087728,0.409661,1.557404,-1.226638,0.428551,-0.173408,-0.060506,-0.101479,0.297092,1.842849
42101,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-1.010111,-2.441040,0.850531,0.573445,-1.149928,-0.173408,-0.060506,-0.101479,-0.526846,-0.737691
42102,0.151040,4.129777,-0.998814,-0.979466,-0.407020,-0.160945,3.088017,-1.059452,0.409661,0.143658,-0.664112,0.428551,-0.173408,-0.060506,-0.101479,3.646696,0.552579
42103,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-0.195983,0.409661,-0.916652,1.585992,-1.149928,-0.173408,-0.060506,-0.101479,-0.641282,-0.737691
42104,-1.753925,-0.313454,-0.998814,-0.278849,2.996268,-0.160945,-0.515173,-1.195140,0.409661,0.497095,0.573445,0.428551,-0.173408,-0.060506,-0.101479,1.061793,-0.737691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70163,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,2.604122,0.409661,0.497095,1.585992,0.428551,-0.173408,-0.060506,-0.101479,1.492610,-0.737691
70164,0.151040,-0.313454,1.259332,-0.278849,-0.407020,-0.160945,-0.515173,-0.282330,0.409661,-1.623525,1.023466,-1.149928,-0.173408,-0.060506,-0.101479,-1.065367,0.552579
70165,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,-0.788076,0.409661,0.850531,0.685950,-1.149928,-0.173408,-0.060506,-0.101479,-0.526846,-0.737691


In [81]:
y_pred = search_4.predict_proba(X_test)
y_pred

array([[0.87758005, 0.12241994],
       [0.9061048 , 0.0938952 ],
       [0.65946925, 0.34053072],
       ...,
       [0.850845  , 0.14915505],
       [0.40858692, 0.5914131 ],
       [0.10764366, 0.89235634]], dtype=float32)

In [82]:
result = pd.DataFrame({'id': X_test.index, 'booking_status':y_pred[:, 1]})
result

,id,booking_status
0,42100,0.122420
1,42101,0.093895
2,42102,0.340531
3,42103,0.097873
4,42104,0.591413
...,...,...
28063,70163,0.865216
28064,70164,0.093895
28065,70165,0.149155
28066,70166,0.591413


In [83]:
result.to_csv('halving_search_4.csv', index=False)

# Conclusion
None of the ensembles tried so far surpassed XGBoost results on the test set (according to the Kaggle public leader board).